<a href="https://colab.research.google.com/github/wuronald/F1L/blob/main/Copy_of_240701_kinker_anndata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages:
Some packages are missing from Google Collab. Let's install them.

In [ ]:
!pip install anndata
!pip install scanpy
!pip install seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.9 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=3c398a1e4a4d4957e3710ba2a5f7333f337a66913f8633c491961f146a98b95a
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


# Load Packages

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
import anndata
import scanpy as sc
import seaborn as sns
import gzip
import shutil

# Check memory use

In [ ]:
!pip install psutil

import os
import psutil

def process_memory():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss


# Download Data
Let's download the raw counts matrix and metadata that was deposited by Kinker et al at two locations: GEO and Broad's Single Cell Portal (SCP).

When looking at GEO, specifically [GSE157220](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE157220), we find supplementary files that correspond to CPM data and an excel spreadsheet with the pool composition. This is not what we need.

Fortunately, visiting SCP at [SCP542](https://singlecell.broadinstitute.org/single_cell/study/SCP542/pan-cancer-cell-line-heterogeneity#study-download) we find the files we are looking for:
1. Metadata.txt
2. UMIcount_data.txt



In [ ]:
# Download CPM counts matrix
# !wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE157nnn/GSE157220/suppl/GSE157220%5FCPM%5Fdata.txt.gz

# Extract
# with gzip.open('/content/GSE157220_CPM_data.txt.gz', 'rb') as f_in:
#     with open('/content/GSE157220_CPM_data.txt', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# Download files from SCP
#!apt-get install ca-certificates
#!apt-get update && apt-get install -y openssl

#!curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP542&auth_code=upqk8ymI&directory=all&context=study"

#!curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP542&auth_code=upqk8ymI&directory=all&context=study" -o content/kinker_data/cfg.txt; curl -K content/kinker_data/cfg.txt && rm content/kinker_data/cfg.txt

#curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/site/studies/SCP542/download?filename=pan-cancer-cell-line-heterogeneity%2FMetadata.txt"

#!wget -P content/kinker_data https://singlecell.broadinstitute.org/single_cell/data/public/SCP542/pan-cancer-cell-line-heterogeneity?filename=Metadata.txt
#!wget -P content/kinker_data https://singlecell.broadinstitute.org/single_cell/data/public/SCP542/pan-cancer-cell-line-heterogeneity?filename=UMIcount_data.txt

Could not get the `curl` command to work in `colab` to download the files programmatically. Even tried the bulk download url provided by the bulk download button in the SCP, which provides a 30 min download window with a token.

I opted to manually download the files from SCP and upload to Google Drive. Since Google Drive is integrated with Google Colab, we can mount the drive and easily have access to the files in this notebook.

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check contents of mounted google drive.

In [ ]:
# Check contents of mounted google drive
%cd /content/drive/MyDrive/F1L/data/kinker/
!ls -ltrh
# copy contents to VM
!cp -r /content/drive/MyDrive/F1L/data/kinker/ /content/data

/content/drive/MyDrive/F1L/data/kinker
total 5.6G
-rw------- 1 root root 3.3G Jul 31 01:58 UMIcount_data.txt
-rw------- 1 root root 8.5M Jul 31 01:59 Metadata.txt
-rw------- 1 root root 2.3G Jul 31 19:52 240701_kinker_anndata.h5ad


# Load data

In [ ]:
# set path to mounted gdrive
gdrive_path = '/content/drive/MyDrive/F1L/data/kinker/'
print(gdrive_path)

# get current working directory
%cd /content
cwd = os.getcwd()
print(cwd)

# read in metadata
meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t')
meta.drop([0], axis=0, inplace=True)  # drop first row (shows data types for each column)
meta.rename(columns={'NAME':'CellID', 'Cell_line':'CellLine', 'Pool_ID':'Pool', 'Cancer_type':'Indication'}, inplace=True) # rename columns
meta

/content/drive/MyDrive/F1L/data/kinker/
/content
/content


<ipython-input-6-dd48a83446db>:11: DtypeWarning: Columns (4,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t')


,CellID,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,...,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
1,AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,NaN,NaN,NaN,NaN,0.166,...,-0.935,-0.935,0.13,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
2,AACGTTGTCACCCGAG-1-18,NCIH2126_LUNG,18,Lung Cancer,5200,NaN,NaN,NaN,NaN,-0.213,...,-1.027,-1.027,0.066,1.049,1.267,0.252,1.299,1.61,0.624,-0.048
3,AACTGGTAGACACGAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4004,NaN,NaN,NaN,NaN,-0.101,...,-0.677,-0.677,0.304,0.822,2.401,0.141,0.451,1.225,-0.795,0.064
4,AACTGGTAGGGCTTGA-1-18,NCIH2126_LUNG,18,Lung Cancer,4295,NaN,NaN,NaN,NaN,-0.014,...,-0.735,-0.735,0.094,0.834,2.282,0.15,0.267,0.892,-0.238,1.118
5,AACTGGTAGTACTTGC-1-18,NCIH2126_LUNG,18,Lung Cancer,4842,NaN,NaN,NaN,NaN,0.006,...,-0.821,-0.821,0.034,0.96,1.4,-0.012,-0.276,-0.428,0.267,0.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53509,c4722,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3343,NaN,NaN,NaN,NaN,0.018,...,-0.505,-0.505,1.657,1.583,3.85,0.539,0.473,0.544,-1.079,-1.349
53510,c4724,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6977,NaN,NaN,NaN,NaN,-0.098,...,-0.876,-0.876,0.669,1.086,3.046,0.799,0.49,1.319,-0.37,0.057
53511,c4731,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6638,NaN,NaN,NaN,NaN,-0.112,...,-0.112,-0.112,0.61,0.693,2.289,0.65,0.729,1.143,-0.508,0.501
53512,c4735,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4052,NaN,NaN,NaN,NaN,-0.244,...,1.981,1.981,0.523,-0.309,0.267,0.822,1.049,0.777,0.296,-0.936


In [ ]:
# Check memory before
mem_before = process_memory()

# Your code chunk here

# the first three rows are CellID, CellLine, and Pool
# only need the first row, CellID
pd.read_csv(cwd+'/data/UMIcount_data.txt', nrows=10, sep='\t', header=None)


# Check memory after
mem_after = process_memory()

print(f"Memory consumed: {mem_after - mem_before} bytes")

Memory consumed: 33742848 bytes


In [ ]:
# extract cell barcodes from first row
counts_cellid = pd.read_csv(cwd+'/data/UMIcount_data.txt', nrows=1, sep='\t', header=None)
counts_cellid = counts_cellid.transpose() # transpose into vertical df
counts_cellid.drop([0], inplace=True) # remove first row
counts_cellid

,0
1,AAACCTGAGACATAAC-1-18
2,AAACCTGCACAACGCC-1-18
3,AAACCTGCAGACAAGC-1-18
4,AAACCTGCAGCTCGAC-1-18
5,AAACCTGCATGGATGG-1-18
...,...
56978,c4788
56979,c4789
56980,c4793
56981,c4800


In [ ]:
%time
# Check memory before
mem_before = process_memory()

# read in raw UMI counts matrix
counts = pd.read_csv(cwd+'/data/UMIcount_data.txt', sep='\t', skiprows=3, header=None, index_col=0) # skip first three rows: CellID, CellLine, and Pool
counts = counts.transpose() # transpose matrix such that columns are genes and rows are cells
counts

# Check memory after
mem_after = process_memory()

print(f"Memory consumed: {mem_after - mem_before} bytes")

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.54 µs
Memory consumed: 33598021632 bytes


In [ ]:

counts.index = counts_cellid[0] # set the index to the barcodes/cellid
counts.index.name = None # remove index name
a = counts.index.isin(meta['CellID']) # check if index is in metadata
counts = counts[a] # subset counts matrix to match metadata barcodes
counts



,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,...,PRICKLE4,RABL6,RAET1E-AS1,RGS5,SERPINA3,SPATA13,TBC1D26,TIMM10B,TMBIM4,TMEM256-PLSCR3
AAACCTGAGACATAAC-1-18,0,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,0,1,0,0
AAACCTGCACAACGCC-1-18,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
AAACCTGCAGACAAGC-1-18,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0
AAACCTGCAGCTCGAC-1-18,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCTGCATGGATGG-1-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,4,4,0
c4789,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
c4793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
c4800,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


Recall in the `Metadata.txt`, we observed 53513 rows. In otherwords, there are 53513 cells in which we have metadata for. We then check how many rows or cell barcodes we have from the `UMIcount_data.txt` UMI matrix. Apparently, there are 56982 rows, suggesting we have more cells in the UMI matrix compared to the metadata. Thus, we subset the imported UMI matrix and keep the rows/barcodes that match the cell barcodes contained in the `Metadata.txt`

In [ ]:
meta = meta.set_index('CellID') # set index to CellID
meta = meta.reindex(index=counts.index) # reindex to match counts matrix
meta

,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,EMTI_score,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,NaN,NaN,NaN,NaN,0.166,-0.045,-0.935,-0.935,0.13,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
AAACCTGCACAACGCC-1-18,SW579_THYROID,18,Thyroid Cancer,5021,NaN,NaN,SW579_THYROID_1,NaN,-0.056,0.776,0.953,0.953,-0.266,-0.334,-1.125,-0.039,-0.243,-0.642,-0.173,1.365
AAACCTGCAGACAAGC-1-18,C32_SKIN,18,Skin Cancer,3047,NaN,NaN,NaN,NaN,1.092,0.617,-0.034,-0.034,0.318,0.57,-0.165,0.074,0.25,0.096,-0.367,-1.135
AAACCTGCAGCTCGAC-1-18,SW579_THYROID,18,Thyroid Cancer,2765,NaN,NaN,SW579_THYROID_1,NaN,-0.601,1.038,1.952,1.952,0.341,-0.253,-0.552,0.921,2.876,1.645,0.226,0.469
AAACCTGCATGGATGG-1-18,NCIH446_LUNG,18,Lung Cancer,2064,NaN,NaN,NaN,NaN,-0.251,-0.325,0.258,0.258,-0.044,-1.256,-0.367,-0.317,0.79,1.925,0.138,-0.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,5929,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,NaN,-0.317,-0.39,0.023,0.023,-0.1,-0.604,-0.358,-0.123,0.067,0.804,0.135,0.264
c4789,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3531,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,0.238,1.176,2.094,2.094,0.014,-0.045,-0.05,0.214,0.238,-0.514,-1.021,-1.243
c4793,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4029,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_3,NaN,NaN,-0.164,-0.163,-0.405,-0.405,0.01,-0.008,0.045,0.265,0.054,-0.466,0.345,0.804
c4800,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4009,SCC9_UPPER_AERODIGESTIVE_TRACT_1,SCC9_UPPER_AERODIGESTIVE_TRACT_1,SCC9_UPPER_AERODIGESTIVE_TRACT_4,SCC9_UPPER_AERODIGESTIVE_TRACT_1,0.205,0.217,0.833,0.833,-0.257,-0.856,-0.593,0.152,0.022,0.684,0.684,0.624


In [ ]:
counts.index.equals(meta.index) # double check to see if indices match

True

# Create AnnData object from imported data
Here we supply the above counts and metadata dataframes that we briefly cleaned and transposed, such that their cellular barcodes are matching and are indexed.

In [ ]:
adata = anndata.AnnData(X=scipy.sparse.csr_matrix(counts),
                        obs=meta,
                        var=counts.columns.to_frame())
del counts # remove counts matrix from memory
adata.var.drop(columns=[0], inplace=True)
adata.var.index.name = None
adata

AnnData object with n_obs × n_vars = 53513 × 30314
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score'

In [ ]:
adata.obs['CellLine'] = adata.obs['CellLine'].astype(str)
adata.obs['Pool'] = adata.obs['Pool'].astype(str)
adata.obs['Indication'] = adata.obs['Indication'].astype(str)
adata.obs['Genes_expressed'] = adata.obs['Genes_expressed'].astype(int)
adata.obs['Discrete_cluster_minpts5_eps1.8'] = adata.obs['Discrete_cluster_minpts5_eps1.8'].astype(str)
adata.obs['Discrete_cluster_minpts5_eps1.5'] = adata.obs['Discrete_cluster_minpts5_eps1.5'].astype(str)
adata.obs['Discrete_cluster_minpts5_eps1.2'] = adata.obs['Discrete_cluster_minpts5_eps1.2'].astype(str)
adata.obs['CNA_subclone'] = adata.obs['CNA_subclone'].astype(str)
adata.obs['SkinPig_score'] = adata.obs['SkinPig_score'].astype(float)
adata.obs['EMTI_score'] = adata.obs['EMTI_score'].astype(float)
adata.obs['EMTII_score'] = adata.obs['EMTII_score'].astype(float)
adata.obs['EMTIII_score'] = adata.obs['EMTIII_score'].astype(float)
adata.obs['IFNResp_score'] = adata.obs['IFNResp_score'].astype(float)
adata.obs['p53Sen_score'] = adata.obs['p53Sen_score'].astype(float)
adata.obs['EpiSen_score'] = adata.obs['EpiSen_score'].astype(float)
adata.obs['StressResp_score'] = adata.obs['StressResp_score'].astype(float)
adata.obs['ProtMatu_score'] = adata.obs['ProtMatu_score'].astype(float)
adata.obs['ProtDegra_score'] = adata.obs['ProtDegra_score'].astype(float)
adata.obs['G1/S_score'] = adata.obs['G1/S_score'].astype(float)
adata.obs['G2/M_score'] = adata.obs['G2/M_score'].astype(float)

In [ ]:
# pre-processing: filter adata object for min_cells and min_genes
# These functions directly modify the object in-place (default)
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.filter_cells(adata, min_genes=200)
adata

AnnData object with n_obs × n_vars = 53513 × 23081
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score', 'n_genes'
    var: 'n_cells'

Brief filtering using scanpy reduced `n_vars` from 30314 to 23081, whereas n_obs was unchanged. This suggests that the genes was reduced, perhaps genes that had less than 10 cells were removed from further analysis.

In [ ]:
adata.write(gdrive_path+'/240701_kinker_anndata.h5ad')